# 打分（能量）函数和能量项 -- Score Function & Energy Terms

@Author: Jian Huang

@email: jian.huang@xtalpi.com

### Introduction

这一章的内容是Rosetta中至关重要的核心 -- **能量函数**和**能量项**

我们知道，Rosetta擅长进行生物大分子蛋白的构象优化，快速在广阔的构象空间中，通过**蒙特卡洛-模拟退火**的核心算法，搜索较优、能量较低的稳定构象。

这里对其过程进行简要介绍。在构象能量优化的过程中，我们首先必须要有对当前构象进行评价的**能量函数**，也称为**打分函数**。能量函数中会对该构象的能量进行数值评价，在三维空间中的由各个原子笛卡尔坐标构成的构象信息，其能量既包含了原子之间成键键能、键角、二面角等“*bonded energy terms*”，也需要包含描述非共价相互作用的其他能量项“*non-bonded terms*”（比如氢键、Lennard-Jones相互作用、静电作用等）。

可以看到，一个能量函数在构象优化过程中至关重要，不管是该能量函数的各个能量项的描述形式还是其中的参数设置，对我们实际进行构象优化有直接的影响。

***

可以想象，当我们使用Rosetta对构象进行了某些随机操作，对应的可以用选取的能量函数（默认的能量函数是REF2015）计算出各个能量项的值和他们的加和值得到的总能量值。在Rosetta中这些随机操作可以看作在广阔的构象空间中漫步，目的是为了走到能量面的“洼地”或“山谷”，这种漫步的过程就涉及到蒙特卡洛-模拟退火算法。

<center><img src="./img/Monte-Carlo-algorithm.jpg" width="800" height="400" align=center /><center/>


**蒙特卡洛-模拟退火算法**可以简单理解为，当我们对构象进行随机操作后，发现计算的总能量值比未操作前低，则以百分之一百的概率接受该构象；相反，当操作后构象能量值高于未操作前的构象能量值，以一定的概率接受该操作（尽管能量升高了！）。而这种概率由一个叫“蒙特卡洛准则，Metropolis criterion”。其数学表示如下：

<center><img src="./img/Metropolis_Criterion.jpg" width="800" height="400" align=center /><center/>

***

*思考1：为什么要使用蒙特卡洛准则，它对于构象搜索有什么独特的优势？*



想了解更多算法内容，推荐阅读：
https://www.cnblogs.com/haimishasha/p/9795592.html
https://zhuanlan.zhihu.com/p/37121528

### 1. Energy Function in PyRosetta

In [1]:
from pyrosetta import *
pyrosetta.init()

PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python36.ubuntu 2020.28+release.8ecab77aa50ac1301efe53641e07e09ac91fee3b 2020-07-07T16:41:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.ubuntu r260 2020.28+release.8ecab77aa50 8ecab77aa50ac1301efe53641e07e09ac91fee3b http://www.pyrosetta.org 2020-07-07T16:41:06
core.init: command: PyRosetta -ex1 -ex2aro -database /home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1780672445 seed_offset=0 real_seed=-1780672445
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-1780672445 RG_type=mt19937



注意运行init()的输出信息，对我们理解pyrosetta帮助也很大，例如下面一句：

>core.init: command: PyRosetta -ex1 -ex2aro -database /home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database

“**-ex1 -ex2aro**”是关于packing过程的flag，其意义是指定packing过程中额外搜索的每个氨基酸残基rotamer的数目.其中ex1指的是所有残基类型chi1角度符合额外chi角cutoff内的采样，ex2aro是芳香类残基的chi2角度符合额外chi角cutoff内的采样。

**-database  路径**  指定了pyrosetta运行过程的数据库，数据库中包含了所有protocol需要的rotamer库，chemical库，**打分函数库**等等，基础但非常重要。

这里我们看一下pyrosetta中内置的打分函数库。进入该database目录，从子目录 scoring/weights/下可以看到很多的weights文件，即内置的、对各个能量项权重已经分配定制好的“**打分函数**”了。

***

我们关注一下打分函数里最常见的一些能量项，如下图：

<center><img src="./img/Energy_Terms.jpg" width="600" height="400" align="center"/></center>


**ATTENTION**
1. 由于rosetta中构象可以表示为全原子描述（Full atom或FA）和质心描述（Centroid或CEN），打分函数也可以分为全原子描述和质心描述的打分函数；
2. 有些能量项可以同时在全原子描述中计算，也可以在质心描述中计算；
3. 关于各个能量项的分解和计算公式，文献中有详细的记录，请参考：《The Rosetta all-atom energy function for macromolecular modeling and design》

DOI： 10.1021/acs.jctc.7b00125

***

>Rosetta中能量项可以分为以下三类：
>1. One Body：通常这类打分项只和单个氨基酸构象有关，比如骨架的二面角，侧链的rotamer构象等；
>2. Two Body：这类打分项与两个氨基酸有关，比如范德华力相互作用，静电相互作用；
>3. Whole Body：从整体几何性质或其他的指标考虑蛋白质的能量，如蛋白质的回旋半径，二级结构组成等可统计的量。

<center><img src="./img/Energy_terms_classification.jpg" width="600" height="400" align="center"/></center>

***

这里给出一些能量项的基本解释，便于大家理解：

| Energy terms | 说明 | 
| --- | --- | 
| fa_atr| 不同残基的原子之间互相吸引的Lennard-Jones作用力，支持标准和非标准氨基酸类型 | 
| fa_rep | 不同残基的原子之间互相排斥的Lennard-Jones作用力，支持标准和非标准氨基酸类型 |
| fa_sol | Lazaridis-Karplus溶剂化能，支持标准和非标准氨基酸类型 |
| fa_intra_rep | 同一残基内原子之间互相排斥的Lennard-Jones作用力，支持标准和非标准氨基酸类型 |
| fa_elec | 库伦静电作用，其介电常数按距离衰减，支持标准和非标准氨基酸类型 |
| pro_close | 脯氨酸闭环能量和前一个残基的psi角能量，支持D和L型脯氨酸氨基酸
| hbond_sr_bb | 骨架-骨架间的在一级序列中靠近的氢键，所有氢键相关能量项均支持标准和非标准氨基酸类型 |
| hbond_lr_bb | 骨架-骨架一级序列中较远的氢键 |
| hbond_bb_sc | 侧链-骨架间的氢键 |
| hbond_sc | 侧链-侧链间的氢键 |
| dslf_fa13 | 二硫键几何能，支持D和L型半胱氨酸 |
| rama | 拉式图统计项的倾向性，支持20中天然氨基酸以及其对映体 |
| omega | 控制稳定的肽键平面所施加的谐振限制 |
| fa_dun | 从Dunbrack对rotamer库的统计中得到的侧链rotamer的内能 |
| p_aa_pp | 主链二面角的概率统计 |
| ref | 解折叠自由能项 |
| lk_ball_wtd |  各向异性的溶解自由能, 假设水分子非均匀分布，用于评估能极性原子 |
| rama_prepro | 骨架二面角倾向性，考虑前一个氨基酸是否为脯氨酸 |

In [2]:
# 初始化一个最常用的REF2015的打分函数对象
my_scorefxn = create_score_function('ref2015')

core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 981 residue types
core.chemical.Gl

In [3]:
# 打印定义的打分函数的详细内容，包括权重、各个能量选项设置等。
print(my_scorefxn)

ScoreFunction::show():
weights: (fa_atr 1) (fa_rep 0.55) (fa_sol 1) (fa_intra_rep 0.005) (fa_intra_sol_xover4 1) (lk_ball_wtd 1) (fa_elec 1) (pro_close 1.25) (hbond_sr_bb 1) (hbond_lr_bb 1) (hbond_bb_sc 1) (hbond_sc 1) (dslf_fa13 1.25) (omega 0.4) (fa_dun 0.7) (p_aa_pp 0.6) (yhh_planarity 0.625) (ref 1) (rama_prepro 0.45)
energy_method_options: EnergyMethodOptions::show: aa_composition_setup_files: 
EnergyMethodOptions::show: mhc_epitope_setup_files: 
EnergyMethodOptions::show: netcharge_setup_files: 
EnergyMethodOptions::show: aspartimide_penalty_value: 25
EnergyMethodOptions::show: etable_type: FA_STANDARD_DEFAULT
analytic_etable_evaluation: 1
EnergyMethodOptions::show: method_weights: ref 1.32468 3.25479 -2.14574 -2.72453 1.21829 0.79816 -0.30065 2.30374 -0.71458 1.66147 1.65735 -1.34026 -1.64321 -1.45095 -0.09474 -0.28969 1.15175 2.64269 2.26099 0.58223
EnergyMethodOptions::show: method_weights: free_res
EnergyMethodOptions::show: unfolded_energies_type: UNFOLDED_SCORE12
EnergyMeth

In [4]:
# 相应地，我们创建一个标准的Centroid打分函数，用于对质心描述的构象进行打分
my_scorefxn_cen = create_score_function('cen_std')

basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/env_log.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/cbeta_den.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/pair_log.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/cenpack_log.txt


In [5]:
# 这里，我们依然还是使用clean后的1ubq_clean.pdb作为例子

from pyrosetta import pose_from_pdb
from pyrosetta.rosetta.protocols.simple_moves import SwitchResidueTypeSetMover

initial_pose = pose_from_pdb("./data/1ubq_clean.pdb")

pose_fullatom = Pose()
pose_fullatom.assign(initial_pose)

# 使用REF2015能量函数对全原子描述的构象打分
my_scorefxn(pose_fullatom)

core.import_pose.import_pose: File './data/1ubq_clean.pdb' automatically determined to be of type PDB
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/home/huangjian/miniconda3/envs/biodesign/lib/python3.6/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.6-linux-x86_64.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbr

32.66393846891525

### 练习一
1. 请重新初始化一个1ubq pose对象(让我们采用clone的复制方法，而不是assign)，转化原始pose为质心描述
2. 使用my_scorefxn_cen能量函数进行打分

**HINT**:创建SwitchResidueTypeSetMover，对pose进行操作

输出值应约为 -11.369


### 思考
1. 我们提到，rosetta中全原子描述和质心描述需要对应的全原子打分函数、质心打分函数进行打分。那么如果将质心打分函数作用到全原子描述上会如何？反之，将全原子打分函数作用到质心描述上又会如何？请使用代码证明你的观点。

2. 是否存在可以兼容质心描述和全原子描述的打分函数？（***）

### 2.定制自己的打分函数

现在我们知道，pyrosetta通过其~database/scoring/weights下的权重文件构建打分函数。它也允许我们自己修改各个打分项对应的权重，以此定制自己的打分函数。

一般而言，我们大多数场景下不太会需要自己修改打分函数的权重，除非你知道你为什么要这么做。这里给出一个简单的设定打分函数的例子。

在rosetta中L-J范式作用被分解为范式吸引项（fa_atr）和范式排斥项(fa_rep)：<br>

<center><img src="./img/LJ-term_fa_atr_fa_rep.jpg" width="500" height="400" align="center"/></center>

我们可以调整fa_atr和fa_rep对该项进行修饰。

In [15]:
from pyrosetta.rosetta.core.scoring import fa_atr, fa_rep
sfxn2 = ScoreFunction()

# 仅仅设置范式排斥和范式吸引项的权重
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)

In [19]:
# 仅计算范式排斥和范式吸引项的得分如下。
sfxn2.show(pose_fullatom)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -397.646    -397.646
 fa_rep                       1.000     103.694     103.694
---------------------------------------------------
 Total weighted score:                     -293.952


### 3. 能量项分解

In [20]:
# 使用show方法，可以查看各个能量项的得分值
my_scorefxn.show(pose_fullatom)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -397.646    -397.646
 fa_rep                       0.550     103.694      57.032
 fa_sol                       1.000     242.952     242.952
 fa_intra_rep                 0.005     355.452       1.777
 fa_intra_sol_xover4          1.000      16.826      16.826
 lk_ball_wtd                  1.000      -8.756      -8.756
 fa_elec                      1.000    -113.094    -113.094
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -18.833     -18.833
 hbond_lr_bb                  1.000     -23.131     -23.131
 hbond_bb_sc                  1.000      -7.389      -7.389
 hbond_sc                     1.000      -1.550      -1.550
 dslf_fa13                    1.250       0.000       0.000
 omega  

In [21]:
# 此外pose对象在经过定义的能量函数计算后，会将最近一次的打分结果储存在pose对象的energy对象下
# 通过energy对象可以获取未取权重的、单个残基的各个能量项打分值

# 查看第一个残基的能量项分解如下：
pose_fullatom.energies().show(1)

core.scoring.Energies: E               fa_atr        fa_rep        fa_sol  fa_intra_repfa_intra_sol_x   lk_ball_wtd       fa_elec     pro_close   hbond_sr_bb   hbond_lr_bb   hbond_bb_sc      hbond_sc     dslf_fa13         omega        fa_dun       p_aa_pp yhh_planarity           ref   rama_prepro
core.scoring.Energies: E(i)   1         -7.46          1.65          3.46          2.18          0.16          0.17         -3.44          0.00          0.00          0.00          0.00          0.00          0.00          0.03          7.86          0.00          0.00          1.66          0.00


In [29]:
# 查看第一个残基的fa_atr得分值
pose_fullatom.energies().residue_total_energies(1)[fa_atr]

-7.457741838514642

#### 练习二
1. 计算1ubq的第十号残基的fa_sol的得分值。

2. 计算1ubq的总得分值里前三名的能量项及其对应值